In [227]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter 
from sklearn.preprocessing import scale
from modAL.density import information_density
from pyod.models.abod import ABOD
np.random.seed(42)
import math
import os
import torch
import time

In [228]:
'''
ft:挑选后的特征
t:特征对应金标准(无)
name:样本名称
c: 特征对应簇
'''
fts = np.load('MAE_feats.npy')
names = np.load('name_list.npy')
print(len(names))

43930


In [229]:
def get_index1(lst=None, item=''):
    tmp = []
    tag = 0
    for i in lst:
        if i == item:
            tmp.append(tag)
        tag += 1
    return tmp

In [230]:
bag = []
for name in names:
    (filepath, filename) = os.path.split(name)
    (filepath, filename) = os.path.split(filepath)

#     bag.append(filename.split('_')[0])
    bag.append(filename)
print(Counter(bag))

Counter({'train': 43930})


In [231]:
bag_index=list(set(bag))
print(bag_index)
len(bag_index)
alpha = 0.40
sel_num = int(np.floor(alpha*43930))
sel_num

['train']


17572

In [232]:
'''
Representation/Outlier
'''
import math

index=[] #index[i]表示第i个聚类对应的index列表
ft_part=[] #ft_part[i]表示第i个聚类对应的特征向量列表
new_index=[] #新列表

for i in bag_index:
    index.append(get_index1(bag, i))
    ft_part.append(torch.Tensor(fts[get_index1(bag, i)]))


for i in range(len(bag_index)):

#     density = information_density(ft_part[i], 'cosine')#cosine/euclidean
    clf = ABOD(method='fast',n_neighbors=10)
    
    start = time.time()
    clf.fit(ft_part[i])
    
    density = clf.decision_scores_

    # get the prediction on the test data
#     y_test_scores = clf.decision_function(ft_part[i])  # outlier scores
#     density = y_test_scores

    centroids = np.argsort(density)[:sel_num] #density为从小到大排列，为代表性；-density为离群性 对于abod库来讲越小越正常
    
    end = time.time()
    print (end-start)
    
    new_index.extend(np.array(index[i])[centroids])

    
ft_sel = fts[new_index]
bag_sel = np.array(bag)[new_index]
name_sel = names[new_index]

from collections import Counter 
print('Resampled dataset:', Counter(bag_sel))
print(len(name_sel))

69.12282133102417
Resampled dataset: Counter({'train': 17572})
17572


In [234]:
'''
Representation/Outlier
'''
import math

index=[] #index[i]表示第i个聚类对应的index列表
ft_part=[] #ft_part[i]表示第i个聚类对应的特征向量列表
new_index=[] #新列表

for i in bag_index:
    index.append(get_index1(bag, i))
    ft_part.append(torch.Tensor(fts[get_index1(bag, i)]))


for i in range(len(bag_index)):
    
    start = time.time()
    density = information_density(ft_part[i], 'cosine')#cosine/euclidean

    # get the prediction on the test data
#     y_test_scores = clf.decision_function(ft_part[i])  # outlier scores
#     density = y_test_scores

    centroids = np.argsort(density)[:sel_num] #density为从小到大排列，为代表性；-density为离群性 对于abod库来讲越小越正常

    end = time.time()
    print (end-start)
    
    new_index.extend(np.array(index[i])[centroids])

    
ft_sel = fts[new_index]
bag_sel = np.array(bag)[new_index]
name_sel = names[new_index]

from collections import Counter 
print('Resampled dataset:', Counter(bag_sel))
print(len(name_sel))

27.938876152038574
Resampled dataset: Counter({'train': 17572})
17572


In [214]:
# y_test_scores

In [215]:
# clf.decision_scores_

In [216]:
'''
注意保存csv的后缀
'''
print(__doc__)
dataframe1 = pd.DataFrame({'image_name':name_sel})
dataframe1.to_csv("./ABOD_N25/MAE_40_ABOD_N25.txt",index=False,sep=' ',header = None)

# dataframe2 = pd.DataFrame(ft_sel)
# dataframe2.to_csv("../FeatureBank/cifar10_1000_ex/1%/cifar10_1000_1%_FB/test_cifar10_1000_1%_fb.csv",
#                   index=False,sep=' ',header = None)





注意保存csv的后缀

